In [7]:
from openff.toolkit import Molecule
molecule = Molecule.from_smiles('c1ccccc1')

from openmmforcefields.generators import GAFFTemplateGenerator
gaff = GAFFTemplateGenerator(molecules=molecule)

from openmm.app import ForceField
forcefield = ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml', 'amber/tip3p_HFE_multivalent.xml')

forcefield.registerTemplateGenerator(gaff.generator)

#from openmm.app import PDBFile
#pdbfile = PDBFile('4W52_h.pdb')

from pdbfixer import PDBFixer

#fix it:
pdb = PDBFixer(str('4W52_h.pdb'))
### Fix messed up residues
pdb.findMissingResidues()
#pdb.findNonstandardResidues()
#pdb.replaceNonstandardResidues() 
pdb.findMissingAtoms()
pdb.addMissingAtoms()    #this adds both missing atoms and residues 

#add hydrogens:
pdb.addMissingHydrogens(7.0)

#print the names of all non-standard residues:
standard_residues = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL', 'HOH']
for residue in pdb.topology.residues():
    if residue.name not in standard_residues:
        print(residue.name)

#make a Modeller object:
from openmm.app import Modeller
modeller = Modeller(pdb.topology, pdb.positions)

#delete EPE:
to_delete = []
for residue in modeller.topology.residues():
    if residue.name == 'EPE':
        to_delete.append(residue)
modeller.delete(to_delete)

#system = forcefield.createSystem(pdb.topology)
system = forcefield.createSystem(modeller.topology)

BNZ
EPE


In [16]:
#integrator:
from openmm import LangevinIntegrator
from openmm import unit
integrator = LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2*unit.femtoseconds)

#simulation:
from openmm.app import Simulation
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

#minimize:
simulation.minimizeEnergy()
from pathlib import Path
input_pdb_file = '4W52_h.pdb'
PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), str( (str(Path(input_pdb_file).stem) + '_minim.pdb')))

#report:
from openmm.app import PDBFile
from openmm.app import DCDReporter
reporting_interval = 10
simulation.reporters.append(DCDReporter((str(Path(input_pdb_file).stem) + '_equil.dcd'), reporting_interval))

#run:
simulation.step(1000)

In [25]:
import mdtraj as md
traj = md.load(
    str((str(Path(input_pdb_file).stem) + '_equil.dcd')),
    top = str( (str(Path(input_pdb_file).stem) + '_minim.pdb'))            #get topology from processed pdb
)

#center the trajectory around the protein (to stop periodic boundary artifacts)
protein = traj.topology.guess_anchor_molecules()
centered_traj = traj.image_molecules(anchor_molecules=protein)

#first let's make copy of trajectory without water:
traj_no_water = centered_traj.atom_slice(centered_traj.topology.select('not water'))

import nglview as nv
view = nv.show_mdtraj(traj_no_water)
view.add_representation('cartoon', selection="protein")
view._set_size('1600px', '800px')
#view.add_representation('ball+stick', selection='water', opacity=0.1)
view

NGLWidget(max_frame=99)

In [22]:
#show with nglview:
import nglview
view = nglview.show_structure_file((str(Path(input_pdb_file).stem) + '_minim.pdb'))
view.add_trajectory((str(Path(input_pdb_file).stem) + '_equil.dcd'))
view


AttributeError: 'str' object has no attribute '__module__'